# Data Prepare

Download, pre-process and analyze data

## Download Data

In [1]:
!aws s3 cp --recursive s3://app-miya/data/ data/

fatal error: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


In [ ]:
#!sed -i "s/^M//g" data/aws_market_label.csv  # type ^M using ctrl(control)+v+m
!sed -i -e ':a' -e 'N' -e '$!ba' -e "s/\n(/(/g" data/aws_market_label.csv

In [ ]:
#!sed -i "s/^M//g" data/aws_user_behavior.csv  # type ^M using ctrl(control)+v+m
#!sed -i -e ':a' -e 'N' -e '$!ba' -e "s/\n(/(/g" data/aws_user_behavior.csv  # Not stable! Use python code instead!

## Pre-Process Data

In [1]:
!head -n 1 data/aws_market_label.csv

02440107010111244,00103,福民,3,超市,114.052,22.5237,[{"POI":"B02F37U19M","name":"天虹商场(福民店)","type":"购物服务;商场;购物中心","direction":"东南","distance":"163.059","location":"114.053041,22.522593","poiweight":"0.529517"},{"POI":"B02F37V58E","name":"招商银行(深圳福民支行)","type":"金融保险服务;银行;招商银行","direction":"南","distance":"123.41","location":"114.052319,22.522630","poiweight":"0.502848"},{"POI":"B02F37SZT4","name":"中国银行(福民支行)","type":"金融保险服务;银行;中国银行","direction":"东","distance":"136.4","location":"114.053322,22.523584","poiweight":"0.428879"},{"POI":"B02F309T28","name":"平安银行(石厦支行)","type":"金融保险服务;银行;平安银行","direction":"西南","distance":"155.886","location":"114.051035,22.522618","poiweight":"0.372843"},{"POI":"B02F30857B","name":"深圳市福田区石厦学校中学部","type":"科教文化服务;学校;中学","direction":"西","distance":"101.893","location":"114.051029,22.523887","poiweight":"0.325091"},{"POI":"B02F37U0ZV","name":"深圳市福田区石厦学校小学部","type":"科教文化服务;学校;小学","direction":"东北","distance":"110.529","location":"114.052753,22.524410","poiweight":"0.35317

In [2]:
fin = open('data/aws_market_label.csv', 'r')
fout = open('data/aws_market_label.tsv', 'w')
while True:
    line = fin.readline()
    if not line:
        break
    params = line.strip().split(',')
    new_line = '\t'.join(params[:7])+'\t'+','.join(params[7:-5]).replace('\t', '')+'\t'+'\t'.join(params[-5:])
    fout.write(new_line+'\n')
fin.close()
fout.close()

In [3]:
!head -n 1 data/aws_user_label.csv

1,088911974338715,3,超市,440000,广东省,440300,深圳市,8ef4453659644a8daea008eaf6c2e656,,,detention,其他,深圳市,广东省,下午,20191009


In [4]:
!head -n 1 data/aws_user_behavior.csv

11,201600022,3,超市,440000,广东省,440300,深圳市,49507b9b1628d97b529b1a2b67cffd8a,90051,天虹便利生活超市邮政六约锦荟,2019071022001400141042394462,,13.0,0.0,0.0,3,19,2,2019-07-10 19:16:50,20190710


In [ ]:
fin = open('data/aws_user_behavior.csv', 'r')
fout = open('data/aws_user_behavior.tsv', 'w')
while True:
    line = fin.readline()
    if not line:
        break
    params = line.strip().split(',')
    if len(params) < 12:
        next_line = fin.readline()
        line = line.strip()+next_line.strip()
        params = line.strip().split(',')
    new_line = '\t'.join(params[:12])+'\t'+','.join(params[12:-8]).replace('\t', '')+'\t'+'\t'.join(params[-8:])
    fout.write(new_line+'\n')
fin.close()
fout.close()

In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['agg.path.chunksize'] = 10000
plt.rcParams['figure.figsize'] = [12, 8]
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:.3f}')

## Market Label

In [7]:
market_label = pd.read_csv('data/aws_market_label.tsv', sep='\t', parse_dates=['dt'], infer_datetime_format=True, header=None, names=['merchant_code', 'market_code', 'market_name', 'format_id', 'format_name', 'longitude', 'latitude', 'poi_info', 'market_scale_label', 'market_oppor_label', 'market_flow_label', 'market_holiday_label', 'dt'])

In [8]:
market_label.head(1)

,merchant_code,market_code,market_name,format_id,format_name,longitude,latitude,poi_info,market_scale_label,market_oppor_label,market_flow_label,market_holiday_label,dt
0,2440107010111244,00103,福民,3,超市,114.052,22.524,"[{""POI"":""B02F37U19M"",""name"":""天虹商场(福民店)"",""type""...",NaN,NaN,NaN,NaN,2019-10-09


In [9]:
market_label.shape

(2355, 13)

In [10]:
market_label.dtypes

merchant_code                    int64
market_code                     object
market_name                     object
format_id                        int64
format_name                     object
longitude                      float64
latitude                       float64
poi_info                        object
market_scale_label              object
market_oppor_label              object
market_flow_label               object
market_holiday_label            object
dt                      datetime64[ns]
dtype: object

In [11]:
market_label_sparse_features = ['merchant_code', 'market_code', 'format_name', 'market_scale_label', 'market_oppor_label', 'market_flow_label', 'market_holiday_label', 'dt']
for sparse_feature in market_label_sparse_features:
    print(sparse_feature+':', len(market_label[sparse_feature].unique()), market_label[sparse_feature].unique()[:5], '...')

merchant_code: 93 [2440107010111244 2440107010116228 2440107020111022 2440107036669000
 2440107039449000] ...
market_code: 1821 ['00103' '00104' '00107' '00109' '00110'] ...
format_name: 13 ['超市' '购物中心' '便利店' '食品' '百货'] ...
market_scale_label: 4 [nan 'C类门店' 'A类门店' 'B类门店'] ...
market_oppor_label: 5 [nan '非重点门店' '成熟门店' '明星门店' '机会门店'] ...
market_flow_label: 6 [nan '低流量门店' '中高流量门店' '高流量门店' '中流量门店'] ...
market_holiday_label: 6 [nan '中高敏感度门店' '高敏感度门店' '中敏感度门店' '中低敏感度门店'] ...
dt: 1 ['2019-10-09T00:00:00.000000000'] ...


In [12]:
market_label_keys = market_label.groupby(['merchant_code', 'market_code']).groups.keys()
print('market_label_keys:', len(market_label_keys))

market_label_keys: 2355


## User Label

In [13]:
user_label = pd.read_csv('data/aws_user_label.csv', sep=',', parse_dates=['dt'], infer_datetime_format=True, header=None, names=['short_id', 'merchant_code', 'format_id', 'format_name', 'province_id', 'province_name', 'city_id', 'city_name', 'buyer_user_id', 'sex', 'age', 'rfm_label', 'pre_payment_label', 'pre_city_label', 'pre_province_label', 'pre_frame_label', 'dt'])

In [14]:
user_label.head(1)

,short_id,merchant_code,format_id,format_name,province_id,province_name,city_id,city_name,buyer_user_id,sex,age,rfm_label,pre_payment_label,pre_city_label,pre_province_label,pre_frame_label,dt
0,1,88911974338715,3,超市,440000,广东省,440300,深圳市,8ef4453659644a8daea008eaf6c2e656,nan,nan,detention,其他,深圳市,广东省,下午,2019-10-09


In [15]:
user_label.shape

(14335794, 17)

In [16]:
user_label.dtypes

short_id                       int64
merchant_code                  int64
format_id                      int64
format_name                   object
province_id                    int64
province_name                 object
city_id                        int64
city_name                     object
buyer_user_id                 object
sex                          float64
age                          float64
rfm_label                     object
pre_payment_label             object
pre_city_label                object
pre_province_label            object
pre_frame_label               object
dt                    datetime64[ns]
dtype: object

In [17]:
user_label_sparse_features = ['short_id', 'merchant_code', 'format_name', 'province_name', 'city_name', 'buyer_user_id', 'sex', 'age', 'rfm_label', 'pre_payment_label', 'pre_city_label', 'pre_province_label', 'pre_frame_label', 'dt']
for sparse_feature in user_label_sparse_features:
    print(sparse_feature+':', len(user_label[sparse_feature].unique()), user_label[sparse_feature].unique()[:5], '...')

short_id: 50 [ 1 11 26 58 60] ...
merchant_code: 93 [  88911974338715   88911659101155   88911831779787 2088431801488876
 2088231354393524] ...
format_name: 13 ['超市' '便利店' '购物中心' '百货' '医药'] ...
province_name: 1 ['广东省'] ...
city_name: 1 ['深圳市'] ...
buyer_user_id: 10111726 ['8ef4453659644a8daea008eaf6c2e656' '74142fbc610c4fc874c6b8f30d5ddfcb'
 '92d6850e4e67d61afb0a0ec11795dcd3' 'bcbc1cfa82543e25a9a6d5877e4cb744'
 'aa2836abe7be2e3f5d8518390f9fa027'] ...
sex: 4 [nan  1.  0.  9.] ...
age: 12 [nan  4.  6.  3.  5.] ...
rfm_label: 6 ['detention' 'cultivate' 'stingy' 'lost' 'champion'] ...
pre_payment_label: 4 ['其他' '支付宝' nan '微信'] ...
pre_city_label: 323 ['深圳市' '天津市市辖区' '广州市' '东莞市' '杭州市'] ...
pre_province_label: 32 ['广东省' '天津市' '浙江省' nan '北京市'] ...
pre_frame_label: 7 ['下午' '晚上' '上午' '中午' '早晨'] ...
dt: 1 ['2019-10-09T00:00:00.000000000'] ...


In [18]:
user_label_keys = user_label.groupby(['short_id', 'format_id', 'buyer_user_id']).groups.keys()
print('user_label_keys:', len(user_label_keys))

user_label_keys: 14335794


## User Behavior

In [ ]:
user_behavior = pd.read_csv('data/aws_user_behavior.tsv', sep='\t', parse_dates=['systemdate', 'dt'], infer_datetime_format=True, dtype={'short_id': 'int64', 'merchant_code': 'int64', 'format_id': 'int64', 'province_id': 'int64', 'city_id': 'int64', 'total_fee': 'float64', 'merchantdiscount': 'float64', 'otherdescount': 'float64', 'hour': 'int64', 'week': 'int64'}, header=None, names=['short_id', 'merchant_code', 'format_id', 'format_name', 'province_id', 'province_name', 'city_id', 'city_name', 'buyer_user_id', 'market_code', 'market_name', 'trade_no', 'goods_detail', 'total_fee', 'merchantdiscount', 'otherdescount', 'payment_id', 'hour', 'week', 'systemdate', 'dt'])

In [ ]:
user_behavior.head(5)

In [ ]:
user_behavior.shape

In [ ]:
user_behavior.dtypes

In [ ]:
user_behavior_sparse_features = ['short_id', 'merchant_code', 'format_name', 'province_name', 'city_name', 'buyer_user_id', 'market_code', 'dt']
for sparse_feature in user_behavior_sparse_features:
    print(sparse_feature+':', len(user_behavior[sparse_feature].unique()), user_behavior[sparse_feature].unique()[:5], '...')